In [1]:
import numpy as np
import os
import scipy
import nibabel as nib
from nibabel.affines import apply_affine
import pymeshlab as mlab
import warnings
try:
    from pymeshlab import Percentage as Percentage
except:
    from pymeshlab import PercentageValue as Percentage
import matplotlib.pyplot as plt
%matplotlib qt
def make_mesh(file):
    """"loads a .nii.gz file and produces a mesh to visualise it.
    Outputs:
    ms: Surface mesh to visualise the volume
    data: Class from nibabel containing all data from the file
    p: array of coordinate points at which there is a non-zero value
    f: array of non-zero values from the image
    """
    # Load file and get data
    data = nib.load(file)
    f = data.get_fdata()
    hdr =data.header
    M = data.affine

    # Obtain voxel coordinates
    nx,ny,nz = f.shape
    i = np.arange(0,nx,1)
    j = np.arange(0,ny,1)
    k = np.arange(0,nz,1)
    [i,j,k] = np.meshgrid(i,j,k)
    sz = list(i.shape)
    sz.append(3)
    index_coords = np.ones(tuple(sz))
    index_coords[:,:,:,0] = i
    index_coords[:,:,:,1] = j
    index_coords[:,:,:,2] = k

    # Obtain RAS coordinates
    p = apply_affine(M,index_coords)
    x = p[:,:,:,0]
    y = p[:,:,:,1]
    z= p[:,:,:,2]
    R = M[0:3,0:3]

    # Use minimum eigenvalue of Affine transformation as length scale for alpha shape
    eig = np.linalg.eigvals(R)
    alpha = np.min(np.max(np.abs(eig)))

    # Filter out non-zero entries
    i,j,k = np.where(f > 0)
    f = f[i,j,k]
    x = x[i,j,k]
    y = y[i,j,k]
    z = z[i,j,k]
    p = np.vstack((x,y,z))
    p = np.array(p).T

    if np.var(f) != 0:
        warnings.warn(f'{file} is not a binary image')


    # Obtain alpha shape
    ms = mlab.MeshSet()
    m = mlab.Mesh(vertex_matrix = p)
    ms.add_mesh(m)
    bbox = ms.get_geometric_measures()['bbox']
    diag = bbox.diagonal()
    ms.generate_alpha_shape(alpha = Percentage(100*alpha/diag),filtering =1)

    return ms,data,p,f

def view_mesh(ms):
    v = ms.current_mesh().vertex_matrix()
    f = ms.current_mesh().face_matrix()
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    ax.plot_trisurf(v[:,0],v[:,1],v[:,2],triangles = f,cmap=plt.cm.Spectral)
    plt.show()

In [2]:
file = 'Data_set/s0011/segmentations/adrenal_gland_left.nii.gz'
ms,data,_,_ = make_mesh(file)

view_mesh(ms)
print(data.header)


<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b''
dim_info        : 0
dim             : [  3 311 311 431   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : uint8
bitpix          : 8
slice_start     : 0
pixdim          : [1.  1.5 1.5 1.5 1.  1.  1.  1. ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 0
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : b''
aux_file        : b''
qform_code      : unknown
sform_code      : aligned
quatern_b       : 0.0
quatern_c       : 0.0
quatern_d       : 0.0
qoffset_x       : -236.54492
qoffset_y       : -45.54492
qoffset_z       : -61.0
srow_x          : [   1.5        0.

In [8]:
ms.save_current_mesh(file+'.ply',binary=False)